# Data Science Experience Hands On Lab1 - Python

The first thing we need to do is connect to a data source and read this into a spark sql dataframe.  Dataframes were new constructs starting in Spark 1.6 and provide a much richer symantex than the prior RDD approach.  

Click on the "Find and Add Data" icon that looks like a series of 1 & 0s in the top right.  This will open up and area on the right hand side of the screen to upload the ACCIENT2007-FullDataSet.csv

Select "browse" to open the local file system and upload this file.  You will see a new file link appear below.

Select the "Insert to code" link that appears underneath the file name and select the "Insert Spark SQL DataFrame" option in the cell below.  This will create a connection to the spark object store where the file is stored and create a resulting Spark SQL DataFrame for you.

In [1]:

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# @hidden_cell
# This function is used to setup the access of Spark to your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def set_hadoop_config_with_credentials_19099026f8df40b6aec4353c7e897e95(name):
    """This function sets the Hadoop configuration so it is possible to
    access data from Bluemix Object Storage using Spark"""

    prefix = 'fs.swift.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', 'cc29768790ec45439a43668592b02f84')
    hconf.set(prefix + '.username', 'd47dac60c7684410842aa453908da4ca')
    hconf.set(prefix + '.password', 'R1o7wzw?37&dHIMq')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', False)

# you can choose any name
name = 'keystone'
set_hadoop_config_with_credentials_19099026f8df40b6aec4353c7e897e95(name)

df_data_1 = sqlContext.read.format('com.databricks.spark.csv')\
  .options(header='true', inferschema='true')\
  .load("swift://DatabricksSpark." + name + "/ACCIDENT2007-FullDataSet.csv")
df_data_1.take(5)


[Row(STATE=1, COUNTY=73, MONTH=1, DAY=2, HOUR=23, MINUTE=15, VE_TOTAL=1, PERSONS=1, PEDS=0, NHS=0, ROAD_FNC=14, ROUTE=4, SP_JUR=0, HARM_EV=42, MAN_COLL=0, REL_JUNC=1, REL_ROAD=4, TRAF_FLO=1, NO_LANES=2, SP_LIMIT=40, ALIGNMNT=2, PROFILE=1, PAVE_TYP=2, SUR_COND=1, TRA_CONT=0, T_CONT_F=0, HIT_RUN=0, LGT_COND=2, WEATHER1=1, WEATHER2=0, C_M_ZONE=0, NOT_HOUR=23, NOT_MIN=16, ARR_HOUR=23, ARR_MIN=20, HOSP_HR=0, HOSP_MN=0, SCH_BUS=0, CF1=0, CF2=0, CF3=0, FATALS=1, DAY_WEEK=3, DRUNK_DR=0, ST_CASE=10001, CITY=0, MILEPT=0.0, YEAR=2007, TWAY_ID=u'1493', TWAY_ID2=u'00000000', RAIL=u'0000000', LATITUDE=33272102, LONGITUD=87010454, VE_FORMS=1, WEATHER=1),
 Row(STATE=1, COUNTY=19, MONTH=1, DAY=30, HOUR=13, MINUTE=5, VE_TOTAL=3, PERSONS=4, PEDS=0, NHS=0, ROAD_FNC=3, ROUTE=3, SP_JUR=0, HARM_EV=12, MAN_COLL=5, REL_JUNC=2, REL_ROAD=1, TRAF_FLO=1, NO_LANES=2, SP_LIMIT=40, ALIGNMNT=1, PROFILE=1, PAVE_TYP=2, SUR_COND=1, TRA_CONT=20, T_CONT_F=3, HIT_RUN=0, LGT_COND=1, WEATHER1=1, WEATHER2=0, C_M_ZONE=0, NOT_HO

Now that you have a dataframe object, you can do analysis based on this such as performing correlation analysis on various variables

You can see the correlation between the individual was drunk and if the accident resulted in fatalities by performing a simple pearson correlation on the two variables.  The notebooks provide code assistance by using the tab after a dot notation to see the various functions availabe.


If you get stuck, the code should be df_data_1.corr('DRUNK_DR','FATALS','pearson')

DSX also provides the ability to import your favorite libraries to use within your notebooks, for example Pandas.  

Import the pandas library and use it to convert the Spark DataFrame to a Pandas DataFrame and use the head function to show the top 5 rows values.

    import pandas as pd
    pd_fars = df_data_1.toPandas()
    pd_fars.head()

You can also get summary statistics using the describe function.  This can help you determine missing values and the distribution of your attributes.    

    pd_fars.describe()

Now we want to look at an individual states worth of data.  The Spark DataFrame object supports a filter option to allow you to filter the data based on a column of interest and the resulting value.  

To see a list of the states, the FARS data dictionairy provides a reference to the state codes here - https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/812092 .  Choose your state of interest to filter the list down to your particular state.  Convert the results to a new Panda dataframe and view the first 5 rows data.  Below is a list of some of the state values;

        12 - Florida      11 - DC          13 - Georgia         48 - Texas    39 - Ohio     36 - New York        34 - New Jersey
        53 - Washingon    51 - Virginia    37 - North Carolina  40 - Oklahoma 32 - Neveda   45 - South Carolina  04 - Arizona

The code should look something like the following;

    df_cal=df_data_1.filter(df_data_1['STATE']==6)
    pd_cal=df_cal.toPandas()
    pd_cal.head(5)


We would now like to map out the occurrences for the state of interest but if you look at the LATITUDE and LONGITUD values, these have not been declared as a float.

We can create new columns for a modified version of these fields so that we can map the individual occurances on a map.

Use lamba functions to create a lon and lat column that represents the expected values.

    pd_cal['lat'] = pd_cal['LATITUDE'].map(lambda x: (x * 1.0) / 1000000)
    pd_cal['lon'] = pd_cal['LONGITUD'].map(lambda x: (x * -1.0) / 1000000)
    pd_cal.head(5)

Now that we have the data we need, we can import the brunel package and use it to show a graphical map of the occurances

Use the following code to display the map for your state using the lon and lat values and use PERSONS to display a color scale based on the number of individuals involved in the accident.

import brunel

%brunel map ('CA') + data('pd_cal') x(lon) y(lat) color(PERSONS) tooltip(VE_TOTAL)

Another graphics package that was created by IBM is the pixie dust package.  Pixie dust provides an easy way to visualize the data using various 
charts.  

To import the pixiedust package, you simply need to use and import statement (from pixiedust.display import *)

Once imported, you bring up the interactive display area by using the display function on your dataset - for example display(df_data_1).

The initial display is a table view of the dataframe.  

Switch views to the pie chart by selecting the middle charting drop down menu at the top left of the display area. This will display a pie chart of the count of accidents by state along with a percentage.  You can view and modify the options used for the display by selecting the paint brush icon at the bottom left of the display area (note this may be invisible until you hover near the area with your mouse).  If you change the value to city instead of state, you will see a busier graph.

You can switch to a different dataframe at anytime by changing the value in the display parameter and rerunning the cell.  

Change the display to use the california data display(df_cal).

select the histogram chart from the drop down and then modify the values to contain city.  

At this point we are ready to start our first pass at building a predictive model.  In this example we will use the statsmodel.formula.api package.  

Use the following code to build a basic linear regression.

    import statsmodels.formula.api as sm

    result = sm.ols(formula="FATALS ~ VE_TOTAL + PERSONS + WEATHER + VE_FORMS", data=pd_cal).fit()
    print result.params

To see a summary of all the results, use print result.summary().

As you can see from the results, this is not a good model at all.  Select a different set of attributes to see if you can improve the R-squared value.